# 2021.02.16

**-Use AlexNets, but also gonna try with ResNets & VGGNets (template prepared) ++ R-CNN**   
**-Normalization should be added**

<h3><span style = "background-color : red">Color Test</span></h3><br>
    - <strong>Using AlexNet</strong><br>
    - <strong>Using color image data</strong>

<h3>Alert!</h3>
- <strong><u>If lr (Learning Rate) is small and decay is big</u>,<br>1) val_loss easily converges to 0 and<br>2) learning ends so fast that ACC never gets better (0.5, 1, asf.. - extreme value)<strong>

In [11]:
import warnings
warnings.filterwarnings(action = "ignore")

import pickle
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization



c_F = pickle.load(open("color_train_F.pickle", "rb"))
c_T = pickle.load(open("color_train_T.pickle", "rb"))


x = np.concatenate( (c_F, c_T), axis = 0 )


y = [1 for i in range(len(c_F))] + [0 for i in range(len(c_T))]

X = []

for rows in x:
    tmp = cv2.resize(rows, (227, 227))
    X.append(tmp)
    
X = np.array(X)
X = X.reshape(-1, 227, 227, 3)
y = np.array(y)

nums = [i for i in range(4800)]

import random

random.seed(4)
random.shuffle(nums)

X = X[nums]
y = y[nums]

INPUT_SHAPE = (227, 227, 3)

model = Sequential()

# 1st Convolutional Layer
model.add( Conv2D(filters=96, input_shape=(INPUT_SHAPE), kernel_size=(11,11), strides=(4,4), padding='valid') )
model.add( Activation('relu') )

# Max Pooling
model.add( MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid') )

# 2nd Convolutional Layer
model.add( Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='same') )
model.add( Activation('relu') )

# Max Pooling
model.add( MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid') )

# 3rd Convolutional Layer
model.add( Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same') )
model.add( Activation('relu') )

# 4th Convolutional Layer
model.add( Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same') )
model.add( Activation('relu') )

# 5th Convolutional Layer
model.add( Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same') )
model.add( Activation('relu') )

# Max Pooling
model.add( MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid') )



########################## Passing it to a Fully Connected layer ##########################
model.add( Flatten() )

K = INPUT_SHAPE[0] * INPUT_SHAPE[1] * INPUT_SHAPE[2]

# 1st Fully Connected Layer
model.add( Dense(4096, input_shape=( K, )) )
model.add( Activation('relu') )
# Add Dropout
model.add( Dropout(0.4) )

# 2nd Fully Connected Layer
model.add( Dense(4096) )
model.add( Activation('relu') )
# Add Dropout
model.add( Dropout(0.4) )

# 3rd Fully Connected Layer
model.add( Dense(1000) )
model.add( Activation('relu') )
# Add Dropout
model.add( Dropout(0.4) )

# Output Layer
model.add( Dense(2) )
model.add( Activation('softmax') )
          
opt = tf.keras.optimizers.Adam(lr = 1e-6)

model.compile(loss = "sparse_categorical_crossentropy", # or  # binary_crossentropy : Dense(1) & Relu
             optimizer = opt,
             metrics = ['accuracy'])

model.summary()

model.fit(X, y, batch_size = 8, epochs = 10, validation_split = 0.1)  # batch_size

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_9 (Activation)    (None, 55, 55, 96)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
activation_10 (Activation)   (None, 27, 27, 256)       0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 13, 13, 384)      

<h3><span style = "background-color : gray">Gray (<span style = "background-color : red">from Color</span>) Test</span></h3><br>
    - <strong>Using AlexNet</strong><br>
    - <strong>Using gray-scale image data</strong>

In [13]:
import warnings
warnings.filterwarnings(action = "ignore")

import pickle
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization



c_F = pickle.load(open("color_train_F.pickle", "rb"))
c_T = pickle.load(open("color_train_T.pickle", "rb"))

**Change color into gray & resize from color images**

In [14]:
F = []
for i in range(c_F.shape[0]):
    F.append(cv2.resize( cv2.cvtColor(c_F[i], cv2.COLOR_BGR2GRAY), (227, 227) ))

T = []
for j in range(c_T.shape[0]):
    T.append(cv2.resize( cv2.cvtColor(c_T[i], cv2.COLOR_BGR2GRAY), (227, 227) ))
    
F, T = np.array(F), np.array(T)

In [15]:
X = np.concatenate( (F, T), axis = 0 )
X = X.reshape(-1, 227, 227, 1)

y = [1 for i in range(len(F))] + [0 for i in range(len(T))]
y = np.array(y)

nums = [i for i in range(c_F.shape[0] + c_T.shape[0])]

import random

random.seed(4)
random.shuffle(nums)

X = X[nums]
y = y[nums]

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.8)
sess = tf.Session(config = tf.ConfigProto(gpu_options = gpu_options))

INPUT_SHAPE = (227, 227, 1)

model = Sequential()

# 1st Convolutional Layer
model.add( Conv2D(filters=96, input_shape=(INPUT_SHAPE), kernel_size=(11,11), strides=(4,4), padding='valid') )
model.add( Activation('relu') )

# Max Pooling
model.add( MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid') )

# 2nd Convolutional Layer
model.add( Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='same') )
model.add( Activation('relu') )

# Max Pooling
model.add( MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid') )

# 3rd Convolutional Layer
model.add( Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same') )
model.add( Activation('relu') )

# 4th Convolutional Layer
model.add( Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same') )
model.add( Activation('relu') )

# 5th Convolutional Layer
model.add( Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same') )
model.add( Activation('relu') )

# Max Pooling
model.add( MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid') )



########################## Passing it to a Fully Connected layer ##########################
model.add( Flatten() )

K = INPUT_SHAPE[0] * INPUT_SHAPE[1] * INPUT_SHAPE[2]

# 1st Fully Connected Layer
model.add( Dense(4096, input_shape=( K, )) )
model.add( Activation('relu') )
# Add Dropout
model.add( Dropout(0.4) )

# 2nd Fully Connected Layer
model.add( Dense(4096) )
model.add( Activation('relu') )
# Add Dropout
model.add( Dropout(0.4) )

# 3rd Fully Connected Layer
model.add( Dense(1000) )
model.add( Activation('relu') )
# Add Dropout
model.add( Dropout(0.4) )

# Output Layer
model.add( Dense(2) )
model.add( Activation('softmax') )
          
opt = tf.keras.optimizers.Adam(lr = 1e-6) #, decay = 1e-5

model.compile(loss = "sparse_categorical_crossentropy", # or  # binary_crossentropy : Dense(1) & Relu
             optimizer = opt,
             metrics = ['accuracy'])

model.summary()

model.fit(X, y, batch_size = 8, epochs = 10, validation_split = 0.1)  # batch_size

## Epoch doesn't work well?
Refer to this; https://stackoverflow.com/questions/37213388/keras-accuracy-does-not-change